In [205]:
import pytorch_lightning as pl
from matplotlib import pyplot as plt
import seaborn as sns
from cca_zoo.deepmodels import (
    DCCA,
    DCCA_NOI,
    DCCA_SDL,
    BarlowTwins,
    get_dataloaders,
    
)
from cca_zoo.deepmodels.utils import architectures, objectives
from cca_zoo.plotting import pairplot_label
from cca_zoo.data import CCA_Dataset
from cca_zoo.models import CCA
import torch.optim as optim
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np


os.chdir('../raw_data')


In [2]:
os.listdir()

['ALL_GNPS.json',
 'ALL_GNPS_15_12_2021_positive_annotated.pickle',
 'ALL_GNPS_15_12_2021_positive_annotated_classifications.txt',
 'all_ms2ds_embedding.pickle',
 'all_spec_embedding.pickle',
 'data.json',
 'df_ALL_GNPS_15_12_2021_positive_annotated_embeddings_class.pickle',
 'df_classes_all_embeddings.pickle',
 'df_classes_max3_embeddings.pickle',
 'df_classes_max5_embeddings.pickle',
 'df_classes_unique_embeddings.pickle',
 'GNPS_15_12_2021_ms2ds_embeddings.pickle',
 'GNPS_spectra_toconfirm',
 'lightning_logs',
 'max3_mol2vec_embedding.pickle',
 'max3_ms2ds_embedding.pickle',
 'max3_spec_embedding.pickle',
 'max5_mol2vec_embedding.pickle',
 'max5_ms2ds_embedding.pickle',
 'max5_spec_embedding.pickle',
 'model_300dim.pkl',
 'mol2vec_embedding.pickle',
 'ms2ds_embeddings_unique_GNPS_15_12_2021.pickle',
 'ms2ds_model_GNPS_15_12_2021.hdf5',
 'ms2ds_pred_corr',
 'ms2ds_pred_corr_tanimoto',
 'ms2ds_pred_cos.filepart',
 'ms2ds_pred_cos_tanimoto',
 'ms2ds_pred_euc',
 'ms2ds_pred_euc_tanimoto

#### Functions to load and or write to pickle files

In [138]:
class Files:
    def __init__(self, filename):
        self.file = filename
        
    def write_to_file(self, data):
        with open(self.file, 'wb') as f:
            pickle.dump(data, f) 
        return None
    
    def load_pickle(self):
        data = pd.read_pickle(self.file)
        return data
    
    def load_csv(self, sep, usecols=None):
        data = pd.read_csv(self.file, sep=sep, usecols=usecols)
        return data
        
        
    

In [177]:
# load df with all info
path = './df_classes_max3_embeddings.pickle'
df_all = Files(path).load_pickle()#[:20000] #just a subset for scripting
df_all.shape

(41447, 8)

In [178]:
#split into train and test dataset
train_df, test_df = \
train_test_split(df_all, test_size=0.3, random_state=42)#, stratify=df_all['cf_class'])

#Split train dataset into train and validation set
test_df, val_df = train_test_split(test_df, test_size=0.1, random_state=42)


#extract the 2 view, v1 == spectra embeddings, v2==structure embeddings
v1_train, v1_test = np.array([x for x in train_df['ms2ds']]), np.array([x for x in test_df['ms2ds']])
v2_train, v2_test = np.array([x for x in train_df['mol2vec']]), np.array([x for x in test_df['mol2vec']])

# validation
v1_val, v2_val = np.array([x for x in val_df['ms2ds']]), np.array([x for x in val_df['mol2vec']])

In [179]:
# What does CCA dataset do????
train_dataset = CCA_Dataset([v1_train, v2_train])
test_dataset = CCA_Dataset([v1_test, v2_test])
val_dataset = CCA_Dataset([v1_val, v2_val])

train_loader , val_loader = get_dataloaders(train_dataset, val_dataset, batch_size=400, num_workers=6,
                                              drop_last=False)
test_loader = get_dataloaders(test_dataset,batch_size=300, num_workers=6,
                                              drop_last=False)

In [206]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

NameError: name 'model' is not defined

In [209]:
#params
N = len(train_dataset)
latent_dims = 100
epochs = 100
#optimizer = optim.SGD(lr=0.01, momentum=0.9)


print('\n','-'*20,'\n Training CCA\n','-'*20)
cca = CCA(latent_dims=latent_dims).fit((v1_train, v2_train))

# define encoders
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=v1_train.shape[1],layer_sizes=(500,300))
                                     
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=v2_train.shape[1], layer_sizes=(500,300))

# define variant algorithmns
# # 1. DCCA
# dcca = DCCA(
#         latent_dims=latent_dims,
#         encoders=[encoder_1, encoder_2],
#         objective=objectives.CCA,
#         )

# 2. SDL
sdl = DCCA_SDL(latent_dims,optimizer='sgd',
                N=N, encoders=[encoder_1, encoder_2],
                lam=0, lr=0.0001,
                ) # train size

# Define the trainer
trainer = pl.Trainer(
        max_epochs=epochs, log_every_n_steps=1,
        
        callbacks=[pl.callbacks.early_stopping.EarlyStopping(monitor="train/sdl")])# early stopage to reduce overfitting

# train
# print('\n','-'*20,'\n Training DCCA\n','-'*20)
# trainer.fit(dcca, train_loader,val_loader)
# print('\n','-'*20,'\n Training SDL\n','-'*20)
trainer.fit(sdl, train_loader,val_loader)

#transform the views to get z projections



 -------------------- 
 Training CCA
 --------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type       | Params
----------------------------------------
0 | encoders | ModuleList | 611 K 
1 | mse      | MSELoss    | 0     
2 | bns      | ModuleList | 0     
----------------------------------------
611 K     Trainable params
0         Non-trainable params
611 K     Total params
2.447     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [210]:
cca_train_corr = cca.score([v1_train, v2_train])
cca_test_corr = cca.score([v1_test, v2_test])

In [211]:
# dcca_corr = dcca.score(train_loader)
# dcca_corr

In [212]:
# dcca_test_corr = dcca.score(test_loader)
# dcca_test_corr

In [213]:
sdl_train_corr = sdl.score(train_loader)
sdl_train_corr[0:5]

array([0.85095832, 0.76998326, 0.69839776, 0.67845128, 0.62847318])

In [214]:
sdl_test_corr = sdl.score(test_loader)
sdl_test_corr[0:5]

array([0.83986669, 0.77490294, 0.6921549 , 0.68397685, 0.63605468])

In [ ]:
# plot train correlation vs test correlation
x=[x for x in range(latent_dims)]
fig, ax = plt.subplots()
sns.set_style('whitegrid')
#[dcca_corr,dcca_test_corr],
#'DCCA Train', 'DCCA Test',
dfs = [[cca_train_corr, cca_test_corr],\
       [sdl_train_corr,sdl_test_corr]]
for df in dfs:
    sns.lineplot(x=x,y=df[0],ax=ax)
    sns.lineplot(x=x,y=df[1],ax=ax)
plt.legend(labels=[\
                  'CCA Train', 'CCA Test',\
                  'SDL Train', 'SDL Test'], loc=(1.04, 0.5), fontsize=15)
plt.xlabel('Latent Dimensions', fontsize=15)
plt.ylabel('Correlation', fontsize=15)
plt.ylim(0, 1.0)
#plt.legend(loc=(1.04, 0))
plt.xlim(0, latent_dims)
plt.yticks(np.arange(0, 1, step=0.1))
plt.show()

In [108]:
# score test dataset
# check the validation loop warning

In [20]:
v1_train.shape

(29012, 200)